### Imports

In [1]:
import pandas as pd
import glob
import datetime
import os, shutil

### Path set-up

In [21]:
if 'DATA_DIR' not in locals():
    DATA_DIR = './data/'
else:
    print(DATA_DIR)

if os.path.exists(DATA_DIR) and os.path.isdir(DATA_DIR):
    shutil.rmtree(DATA_DIR)
os.makedirs(os.path.dirname(DATA_DIR), exist_ok=True)

./data/


In [ ]:
if 'OUTPUT_DATA_FOLDER' not in locals():
    OUTPUT_DATA_FOLDER = './output/'
else:
    print(OUTPUT_DATA_FOLDER)

if os.path.exists(OUTPUT_DATA_FOLDER) and os.path.isdir(OUTPUT_DATA_FOLDER):
    shutil.rmtree(OUTPUT_DATA_FOLDER)
os.makedirs(os.path.dirname(OUTPUT_DATA_FOLDER), exist_ok=True)

In [ ]:
if 'ELASTIC_INDEX' not in locals():
    ELASTIC_INDEX = 'siren'
else:
    print(ELASTIC_INDEX)

### Import data

In [2]:
# Import Stock Unite Legale data
df_unite_legale = pd.read_csv('https://files.data.gouv.fr/insee-sirene/StockUniteLegale_utf8.zip', compression='zip', dtype=str, usecols=['siren', 
       'dateCreationUniteLegale', 'sigleUniteLegale',
       'prenom1UniteLegale','identifiantAssociationUniteLegale', 'trancheEffectifsUniteLegale', 
       'dateDernierTraitementUniteLegale', 'categorieEntreprise','etatAdministratifUniteLegale',
       'nomUniteLegale', 'denominationUniteLegale', 'categorieJuridiqueUniteLegale',
       'activitePrincipaleUniteLegale', 'economieSocialeSolidaireUniteLegale'])

CPU times: user 1min 3s, sys: 30.9 s, total: 1min 34s
Wall time: 2min 1s


In [3]:
# Rename columns
df_unite_legale = df_unite_legale.rename(columns={
    'dateCreationUniteLegale': 'date_creation_entreprise', 
    'sigleUniteLegale': 'sigle',
    'prenom1UniteLegale': 'prenom',
    'trancheEffectifsUniteLegale': 'tranche_effectif_salarie_entreprise',
    'dateDernierTraitementUniteLegale': 'date_mise_a_jour',
    'categorieEntreprise': 'categorie_entreprise',
    'nomUniteLegale': 'nom',
    'denominationUniteLegale': 'nom_raison_sociale',
    'categorieJuridiqueUniteLegale': 'nature_juridique_entreprise',
    'activitePrincipaleUniteLegale': 'activite_principale_entreprise'
})

CPU times: user 3.09 s, sys: 8.71 s, total: 11.8 s
Wall time: 15.6 s


### Data preprocessing

In [ ]:
 def nom_complet(x):
    if(x['nature_juridique_entreprise'] == '1000'):
        if(x['sigle'] == x['sigle']):
            if((x['prenom'] == x['prenom']) & (x['nom'] == x['nom'])):
                return x['prenom'].lower()+' '+x['nom'].lower()+' ('+x['sigle'].lower()+')'
            else:
                return None
        else:
            if((x['prenom'] == x['prenom']) & (x['nom'] == x['nom'])):
                return x['prenom'].lower()+' '+x['nom'].lower()
            else: 
                return None
    else:
        if(x['sigle'] == x['sigle']):
            if(x['nom_raison_sociale'] == x['nom_raison_sociale']):
                return x['nom_raison_sociale'].lower()+' ('+x['sigle'].lower()+')'
            else:
                return None
        else:
            if(x['nom_raison_sociale'] == x['nom_raison_sociale']):
                return x['nom_raison_sociale'].lower()
            else:
                return None

In [5]:
# Add nom_complet column to df_unite_legale
df_unite_legale['nom_complet'] = df_unite_legale.apply(lambda row: nom_complet(row), axis=1)

CPU times: user 10min 6s, sys: 11.9 s, total: 10min 17s
Wall time: 10min 26s


In [6]:
# Create list of departement zip codes
all_deps = [*'-0'.join(list(str(x) for x in range(0, 10))).split('-')[1:],
    *list(str(x) for x in range(10,20)),
    *['2A','2B'],
    *list(str(x) for x in range(21,95)),
    *'-7510'.join(list(str(x) for x in range(0, 10))).split('-')[1:],
    *'-751'.join(list(str(x) for x in range(10, 21))).split('-')[1:],
    *['']
    ]
# Remove Paris zip code
all_deps.remove('75')

CPU times: user 79 µs, sys: 83 µs, total: 162 µs
Wall time: 90.1 µs


In [22]:
# Upload geo data by departement
for dep in all_deps:
    url = 'https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_'+dep+'.csv.gz'
    print(url)
    df_dep = pd.read_csv(
            url,
            compression="gzip",
            dtype=str,
            usecols=['siren', 'siret',
           'dateCreationEtablissement', 'trancheEffectifsEtablissement',
           'activitePrincipaleRegistreMetiersEtablissement',
           'etablissementSiege',
           'numeroVoieEtablissement',
           'libelleVoieEtablissement',
           'codePostalEtablissement', 'libelleCommuneEtablissement',
            'codeCommuneEtablissement',
            'dateDebut', 'etatAdministratifEtablissement', 'enseigne1Etablissement',
           'activitePrincipaleEtablissement',
           'geo_adresse']
        )
    df_dep = df_dep.rename(columns={
            'dateCreationEtablissement': 'date_creation',
            'trancheEffectifsEtablissement': 'tranche_effectif_salarie',
            'activitePrincipaleRegistreMetiersEtablissement': 'activite_principale_registre_metier',
            'etablissementSiege': 'is_siege',
            'numeroVoieEtablissement': 'numero_voie',
            'typeVoieEtablissement': 'type_voie',
            'libelleVoieEtablissement': 'libelle_voie',
            'codePostalEtablissement': 'code_postal',
            'libelleCommuneEtablissement': 'libelle_commune',
            'codeCommuneEtablissement': 'commune',
            'codeCedexEtablissement': 'cedex',
            'dateDebut_x': 'date_debut_activite', 
            'etatAdministratifEtablissement': 'etat_administratif_etablissement',
            'enseigne1Etablissement': 'enseigne',
            'activitePrincipaleEtablissement': 'activite_principale'
        })
    df_dep.to_csv(DATA_DIR+'geo_siret_'+dep+'.csv',index=False)

https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_01.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_02.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_03.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_04.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_05.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_06.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_07.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_08.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_09.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_10.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_11.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_12.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_13.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_14.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_15.cs

In [23]:
# Get geo data file paths
geo_files = glob.glob(DATA_DIR+"geo_siret*.csv")

CPU times: user 1.45 ms, sys: 1.06 ms, total: 2.51 ms
Wall time: 1.72 ms


In [25]:
geo_files.sort()

In [27]:
# Compute 'nombre d'établissements' per 'siren'
df_out=pd.DataFrame()
for geo_file in geo_files:
    print(geo_file)
    df_geo = pd.read_csv(geo_file,dtype=str)
    df_geo['file'] = geo_file
    df_geo['nombre_etablissements'] = df_geo.groupby(['siren','file'])['siret'].transform('count')
    df_inter = df_geo[['siren','file','nombre_etablissements']]
    df_out = pd.concat([df_out, df_inter])
df_out = df_out.drop_duplicates(keep='first')
df_out2 = df_out[['siren','nombre_etablissements']].groupby(['siren'],as_index=False).sum()
df_unite_legale = pd.merge(df_unite_legale,df_out2,on='siren',how='left')

./data/geo_siret_01.csv
./data/geo_siret_02.csv
./data/geo_siret_03.csv
./data/geo_siret_04.csv
./data/geo_siret_05.csv
./data/geo_siret_06.csv
./data/geo_siret_07.csv
./data/geo_siret_08.csv
./data/geo_siret_09.csv
./data/geo_siret_10.csv
./data/geo_siret_11.csv
./data/geo_siret_12.csv
./data/geo_siret_13.csv
./data/geo_siret_14.csv
./data/geo_siret_15.csv
./data/geo_siret_16.csv
./data/geo_siret_17.csv
./data/geo_siret_18.csv
./data/geo_siret_19.csv
./data/geo_siret_21.csv
./data/geo_siret_22.csv
./data/geo_siret_23.csv
./data/geo_siret_24.csv
./data/geo_siret_25.csv
./data/geo_siret_26.csv
./data/geo_siret_27.csv
./data/geo_siret_28.csv
./data/geo_siret_29.csv
./data/geo_siret_2A.csv
./data/geo_siret_2B.csv
./data/geo_siret_30.csv
./data/geo_siret_31.csv
./data/geo_siret_32.csv
./data/geo_siret_33.csv
./data/geo_siret_34.csv
./data/geo_siret_35.csv
./data/geo_siret_36.csv
./data/geo_siret_37.csv
./data/geo_siret_38.csv
./data/geo_siret_39.csv
./data/geo_siret_40.csv
./data/geo_siret

In [28]:
# Compute 'nombre etablissements ouvert' per 'siren'
df_out=pd.DataFrame()
for geo_file in geo_files:
    print(geo_file)
    df_geo = pd.read_csv(geo_file,dtype=str)
    df_geo = df_geo[df_geo['etat_administratif_etablissement'] == 'A']
    df_geo['file'] = geo_file
    df_geo['nombre_etablissements_ouvert'] = df_geo.groupby(['siren','file'])['siret'].transform('count')
    df_inter = df_geo[['siren','file','nombre_etablissements_ouvert']]
    df_out = pd.concat([df_out, df_inter])
df_out = df_out.drop_duplicates(keep='first')
df_out2 = df_out[['siren','nombre_etablissements_ouvert']].groupby(['siren'],as_index=False).sum()
df_unite_legale = pd.merge(df_unite_legale,df_out2,on='siren',how='left')

./data/geo_siret_01.csv
./data/geo_siret_02.csv
./data/geo_siret_03.csv
./data/geo_siret_04.csv
./data/geo_siret_05.csv
./data/geo_siret_06.csv
./data/geo_siret_07.csv
./data/geo_siret_08.csv
./data/geo_siret_09.csv
./data/geo_siret_10.csv
./data/geo_siret_11.csv
./data/geo_siret_12.csv
./data/geo_siret_13.csv
./data/geo_siret_14.csv
./data/geo_siret_15.csv
./data/geo_siret_16.csv
./data/geo_siret_17.csv
./data/geo_siret_18.csv
./data/geo_siret_19.csv
./data/geo_siret_21.csv
./data/geo_siret_22.csv
./data/geo_siret_23.csv
./data/geo_siret_24.csv
./data/geo_siret_25.csv
./data/geo_siret_26.csv
./data/geo_siret_27.csv
./data/geo_siret_28.csv
./data/geo_siret_29.csv
./data/geo_siret_2A.csv
./data/geo_siret_2B.csv
./data/geo_siret_30.csv
./data/geo_siret_31.csv
./data/geo_siret_32.csv
./data/geo_siret_33.csv
./data/geo_siret_34.csv
./data/geo_siret_35.csv
./data/geo_siret_36.csv
./data/geo_siret_37.csv
./data/geo_siret_38.csv
./data/geo_siret_39.csv
./data/geo_siret_40.csv
./data/geo_siret

In [ ]:
# Merge geo files with above dataframe
for geo_file in geo_files:
    print(geo_file)
    df_geo = pd.read_csv(geo_file,dtype=str)
    df_inter = pd.merge(df_geo,df_unite_legale,on='siren',how='left')
    df_inter2 = df_inter[df_inter['is_siege'] == 'true']
    df_inter.to_csv(OUTPUT_DATA_FOLDER+'siret_'+geo_file.replace(DATA_DIR+'geo_siret_',''), index=False)
    df_inter2.to_csv(OUTPUT_DATA_FOLDER+ELASTIC_INDEX+'_'+geo_file.replace(DATA_DIR+'geo_siret_',''), index=False)